# Merge mat and camera data into one video

In [1]:
import os
import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import numpy as np
import ffmpeg
import PIL
from PIL import Image
from PIL import ImageOps
# from PIL import Resampling
from matplotlib import cm

Magic number for image alignment.

In [2]:
camera_image_width, camera_image_height = 80, 60
camera_image_rescale = 1.5

mat_image_width, mat_image_height = 27, 64
camera_min_value, camera_max_value = 7886.0, 8171.0

Where to find the files we're after for this data fusion, and where to write the results.

In [3]:
camera_path = os.path.join('data', 'camera_data') # Infra Red camera data'
mat_path = os.path.join('data', 'mat_data') # ..\Pressure Mat Data'

# centre_camera_path = os.path.join(camera_path, 'tests', 'subject1', 'Centre 1')
# centre_mat_path = os.path.join(mat_path, 'tests', 'subject1', 'Centre 1')

In [5]:
this_camera_path = os.path.join(camera_path, 'Lyne', 'ADL IR', 'LIRADL2')
this_mat_path = os.path.join(mat_path, 'Lyne', 'ADL Mat', 'LMATADL2')

mat_csv_file = 'LA2.csv'

outfile_name = 'lyne_fall_29_mixed'

# camera_frame_number = 2
# mat_frame_number = 700

Read the timestamp of a camera file.

In [6]:
def camera_date_reader(d):
    try:
        return datetime.datetime.strptime(d, 'frame_%Y-%m-%dT%H_%M_%S.%f.csv')
    except ValueError:
        return datetime.datetime.strptime(d, 'frame_%Y-%m-%dT%H_%M_%S.csv')

Read the timestamp in a mat file.

In [7]:
def mat_date_reader(d):
    try:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S.%f%z')
    except ValueError:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z')

Write camera data to a video.

In [8]:
def vidwrite_camera(fn, images, framerate=8, vcodec='libx264'):
    if not isinstance(images, np.ndarray):
        images = np.asarray(images)
    n,height,width = images.shape
    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='gray', s=f'{width}x{height}')
            .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate,
                    vf="drawtext=fontfile=Arial.ttf: text=%{n}: x=0: y=h-(2*lh): fontcolor=white: box=1: boxcolor=0x00000099"
                   )
            .overwrite_output()
            .run_async(pipe_stdin=True)
    )
    for frame in images:
        process.stdin.write(
            (frame * 255)
                .astype(np.uint8)
                .tobytes()
        )
    process.stdin.close()
    process.wait()

Write mat data to a video.

In [9]:
def vidwrite_mat(fn, images, framerate=30, vcodec='libx264'):
    if not isinstance(images, np.ndarray):
        images = np.asarray(images)
    n,height,width = images.shape
    width += 1
    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgb24', s=f'{width}x{height}')
            .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate,
                    vf="drawtext=fontfile=Arial.ttf: text=%{n}: x=0: y=5: fontcolor=white: box=1: boxcolor=0x00000099"
                   )
            .overwrite_output()
            .run_async(pipe_stdin=True)
    )
    for frame in images:
        mod_frame = np.hstack((frame, np.zeros((64, 1), dtype=frame.dtype)))
        process.stdin.write(
            (cm.hot(mod_frame) * 255)[:, : , :-1]
                .astype(np.uint8)
                .tobytes()
        )
    process.stdin.close()
    process.wait()

Read all the camera files, store them in order (this follows the timestamps)

In [10]:
camera_files = {}
for f in os.listdir(this_camera_path):
    fp = os.path.join(this_camera_path, f)
    if os.path.isfile(fp) and 'ContDrc' not in f:
        tm = camera_date_reader(f).replace(tzinfo=ZoneInfo('Europe/London'))
        camera_files[tm] = fp
        # print(f, tm)
camera_files = {t: camera_files[t] for t in sorted(camera_files)}
camera_files

{datetime.datetime(2024, 7, 29, 11, 6, 56, 153289, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.153289.csv',
 datetime.datetime(2024, 7, 29, 11, 6, 56, 271476, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.271476.csv',
 datetime.datetime(2024, 7, 29, 11, 6, 56, 449475, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.449475.csv',
 datetime.datetime(2024, 7, 29, 11, 6, 56, 561437, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.561437.csv',
 datetime.datetime(2024, 7, 29, 11, 6, 56, 679982, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.679982.csv',
 datetime.datetime(2024, 7, 29, 11, 6, 56, 865400, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/Lyn

Leftover from development.

In [11]:
time_sorted_filenames = [camera_files[t] for t in sorted(camera_files)]
time_sorted_filenames

['data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.153289.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.271476.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.449475.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.561437.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.679982.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.865400.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_56.995387.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_57.114304.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_57.264566.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_57.427227.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_57.558435.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_57.703854.csv',
 'data/camera_data/Lyne/ADL IR/LIRADL2/frame_2024-07-29T11_06_57

Remember on particular filename for when I want to work on just one file.

In [12]:
tfile = time_sorted_filenames[0]

What are the differences in time between successive frames?

In [13]:
ctimes = [t for t in camera_files]
diffs = [b - a for a, b in zip(ctimes, ctimes[1:])]
diffs

[datetime.timedelta(microseconds=118187),
 datetime.timedelta(microseconds=177999),
 datetime.timedelta(microseconds=111962),
 datetime.timedelta(microseconds=118545),
 datetime.timedelta(microseconds=185418),
 datetime.timedelta(microseconds=129987),
 datetime.timedelta(microseconds=118917),
 datetime.timedelta(microseconds=150262),
 datetime.timedelta(microseconds=162661),
 datetime.timedelta(microseconds=131208),
 datetime.timedelta(microseconds=145419),
 datetime.timedelta(microseconds=118556),
 datetime.timedelta(microseconds=148071),
 datetime.timedelta(microseconds=147435),
 datetime.timedelta(microseconds=178166),
 datetime.timedelta(microseconds=88577),
 datetime.timedelta(microseconds=171271),
 datetime.timedelta(microseconds=105068),
 datetime.timedelta(microseconds=169121),
 datetime.timedelta(microseconds=126669),
 datetime.timedelta(microseconds=147702),
 datetime.timedelta(microseconds=123834),
 datetime.timedelta(microseconds=165379),
 datetime.timedelta(microseconds=14

Load the camera data.

Open each file in turn, read it (with `loadtxt`) into a Numpy array, reshape that array into the 60×80 grid of the image, shove the array into a list.

When done, use `stack` to smash the list together into one big 3d array. First index is time, second is short side, third is long side of image.

In [14]:
camera_frames = []
for f in time_sorted_filenames:
    arr = np.loadtxt(f, delimiter=',', usecols=list(range(80)), max_rows=60).reshape((1, 60, 80))
    camera_frames.append(arr)
camera_frames = np.vstack(camera_frames)
camera_frames.shape

(124, 60, 80)

Normalise all the values of the camera data, so they go from 0 to 1.

In [15]:
cmin, cmax = np.min(camera_frames), np.max(camera_frames)
camera_frames = (camera_frames - cmin) / (cmax - cmin)
camera_frames

array([[[0.14089347, 0.13058419, 0.15120275, ..., 0.18900344,
         0.09965636, 0.12027491],
        [0.12027491, 0.13058419, 0.13058419, ..., 0.11683849,
         0.07560137, 0.18900344],
        [0.16494845, 0.20274914, 0.12714777, ..., 0.19243986,
         0.17525773, 0.16494845],
        ...,
        [0.24742268, 0.23367698, 0.2233677 , ..., 0.29553265,
         0.29209622, 0.30927835],
        [0.13745704, 0.22680412, 0.20618557, ..., 0.32646048,
         0.32989691, 0.29209622],
        [0.21993127, 0.21993127, 0.24054983, ..., 0.3024055 ,
         0.28178694, 0.27491409]],

       [[0.17525773, 0.12027491, 0.15120275, ..., 0.19243986,
         0.17869416, 0.13745704],
        [0.21649485, 0.09621993, 0.15120275, ..., 0.14089347,
         0.17525773, 0.12371134],
        [0.20274914, 0.14776632, 0.1580756 , ..., 0.14776632,
         0.22680412, 0.13058419],
        ...,
        [0.25773196, 0.21305842, 0.17182131, ..., 0.24398625,
         0.33676976, 0.35395189],
        [0.1

Read just one file. I think this was leftover from experiments.

In [16]:
tframe = pd.read_csv(tfile, header=None, skipfooter=1, engine='python').drop(80, axis='columns')
tframe.shape

(60, 80)

Read the mat data file, converting the time, keeping the frame number as the index.

In [17]:
mframe = pd.read_csv(os.path.join(this_mat_path,mat_csv_file), 
                     converters={'Timestamp': mat_date_reader},
                    index_col='Frame')
mframe.dtypes

Timestamp           datetime64[ns, UTC+01:00]
Range Min (mmHg)                        int64
Range Max (mmHg)                        int64
0                                       int64
1                                       int64
                              ...            
1723                                    int64
1724                                    int64
1725                                    int64
1726                                    int64
1727                                    int64
Length: 1731, dtype: object

In [18]:
mframe

,Timestamp,Range Min (mmHg),Range Max (mmHg),0,1,2,3,4,5,6,...,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727
Frame,,,,,,,,,,,,,,,,,,,,,
1,2024-07-29 11:06:57.544000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2024-07-29 11:06:57.576000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2024-07-29 11:06:57.609000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2024-07-29 11:06:57.641000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2024-07-29 11:06:57.674000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,2024-07-29 11:07:11.476000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
431,2024-07-29 11:07:11.508000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
432,2024-07-29 11:07:11.541000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Capture the mat and camera times

In [19]:
mat_times = [t.to_pydatetime() for t in mframe['Timestamp']]
mat_times

[datetime.datetime(2024, 7, 29, 11, 6, 57, 544000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 576000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 609000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 641000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 674000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 706000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 738000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 771000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 803000, tzinfo=datetime.timezone(datetime.timedelta(seconds=3

In [20]:
camera_times = list(sorted(camera_files.keys()))
camera_times

[datetime.datetime(2024, 7, 29, 11, 6, 56, 153289, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 56, 271476, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 56, 449475, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 56, 561437, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 56, 679982, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 56, 865400, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 56, 995387, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 114304, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 264566, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 6, 57, 427227, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.

Sanity check: do the times cover a sensible period (on one occasion, there were a couple of camera frames that were about three days later than the others. That led to a long video being created!)

In [21]:
start_time = min(mat_times[0], camera_times[0])
end_time = max(mat_times[-1], camera_times[-1])
start_time, end_time

(datetime.datetime(2024, 7, 29, 11, 6, 56, 153289, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 datetime.datetime(2024, 7, 29, 11, 7, 13, 478405, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')))

Build the structure of the overlaid video.

Go from the `start_time` to the `end_time`, in steps of `frame_delta`. At each step, find the most recent mat or camera time that's less than the `frame_time` and add it to `frame_sources`. Then advance `frame_time`.

You're left with a `dict` of times of frames in the composite video. For each entry in the `dict`, you can find the `mat_time` and `camera_time` to extract the data for that frame of the composite video.

In [22]:
frame_delta = datetime.timedelta(seconds=(1.0 / 30))
frame_time = start_time
frame_sources = {}

current_mat_time = None
current_camera_time = None

while frame_time < end_time:
    while mat_times and mat_times[0] < frame_time:
        current_mat_time = mat_times[0]
        mat_times = mat_times[1:]
    while camera_times and camera_times[0] < frame_time:
        current_camera_time = camera_times[0]
        camera_times = camera_times[1:]
    if not mat_times: current_mat_time = None
    if not camera_times: current_camera_time = None
    frame_sources[frame_time] = {'mat_time': current_mat_time, 'camera_time': current_camera_time}
    frame_time += frame_delta

frame_sources

{datetime.datetime(2024, 7, 29, 11, 6, 56, 153289, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): {'mat_time': None,
  'camera_time': None},
 datetime.datetime(2024, 7, 29, 11, 6, 56, 186622, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): {'mat_time': None,
  'camera_time': datetime.datetime(2024, 7, 29, 11, 6, 56, 153289, tzinfo=zoneinfo.ZoneInfo(key='Europe/London'))},
 datetime.datetime(2024, 7, 29, 11, 6, 56, 219955, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): {'mat_time': None,
  'camera_time': datetime.datetime(2024, 7, 29, 11, 6, 56, 153289, tzinfo=zoneinfo.ZoneInfo(key='Europe/London'))},
 datetime.datetime(2024, 7, 29, 11, 6, 56, 253288, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): {'mat_time': None,
  'camera_time': datetime.datetime(2024, 7, 29, 11, 6, 56, 153289, tzinfo=zoneinfo.ZoneInfo(key='Europe/London'))},
 datetime.datetime(2024, 7, 29, 11, 6, 56, 286621, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): {'mat_time': None,
  'camera_time': datetime.datetime(202

In [23]:
mat_times

[]

Convert the mat data into a numpy array, convert to floats, normalise to range 0-1, and reshape.

In [24]:
mat_frames = mframe.drop(['Timestamp', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis='columns')
mat_frames = (mat_frames.to_numpy().astype(np.float64) / 104).reshape(-1, 64, 27)
mat_frames

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

A couple of functions that convert a camera frame array to an image, and a mat frame array to an image.

In [25]:
def camera_image_for_timestamp(timestamp):
    if timestamp is None:
        return Image.new(mode='RGBA', size=(int(camera_image_width * camera_image_rescale), int(camera_image_height * camera_image_rescale)), color='#000f')
    else:
        f = camera_files[timestamp]
        arr = np.loadtxt(f, delimiter=',', usecols=list(range(80)), max_rows=60).reshape((camera_image_height, camera_image_width))
        arr = (arr - camera_min_value) / (camera_max_value - camera_min_value)
        arr = np.maximum(arr, 0.0)
        arr = np.minimum(arr, 1.0)
        im = Image.fromarray((arr * 255).astype(np.uint8))
        im = im.convert(mode='RGBA')
        return im.resize((int(camera_image_width * camera_image_rescale), int(camera_image_height * camera_image_rescale)))

In [26]:
def mat_image_for_timestamp(timestamp):
    if timestamp is None:
        return Image.new(mode='RGBA', size=(int(camera_image_width * camera_image_rescale), int(camera_image_height * camera_image_rescale)), color='#0000')
    else:
        mat_line = mframe.loc[mframe['Timestamp'] == timestamp]
        mat_line = mat_line.drop(['Timestamp', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis='columns')
        arr = (mat_line.to_numpy().astype(np.float64) / 104).reshape(mat_image_height, mat_image_width)
        mask_arr = (arr > 0.05) * 255
        mask = Image.fromarray(mask_arr.astype(np.uint8))
        arr = (cm.hot(arr) * 255)
        im = Image.fromarray(arr.astype(np.uint8))
        # mask = im.point(lambda p: 255 if p > 5 else 0)
        
        # im = im.convert(mode='RGBA')
        im.putalpha(mask)
# c = corner2_camera.convert('RGBA')
# mask = corner2_camera.point(lambda p: 255 if p > 130 else 0)
# c.putalpha(mask)
# acc.alpha_composite(c)
        
        # im = ImageOps.mirror(im)
        im = ImageOps.flip(im)
        box = Image.new(mode='RGBA', size=(im.width + im.height, im.width + im.height), color='#0000')
        box.alpha_composite(im)
        box = box.rotate(90, center=(mat_image_width, 0), resample=Image.Resampling.NEAREST, fillcolor=(0, 0, 0, 0))
        box = box.crop((im.width, 0, im.width + im.height, im.width + im.height))
        return box
        


# # mc.paste(mat_corners)
# mat_corners_flipped= ImageOps.mirror(mat_corners)
# mc.alpha_composite(mat_corners_flipped)
# mc = mc.rotate(90, center=(27, 0), resample=Image.Resampling.NEAREST, fillcolor=(0, 0, 0, 0))
# mc = mc.crop((mat_corners.width, 0, mat_corners.width + mat_corners.height, mat_corners.width + mat_corners.height))
# # mc = ImageOps.flip(mc)
# # mc.width, mc.height
# mc

A function to put the two together. At each timestamp in `timestamps`, build the camera and mat images, overlay them into one with `alpha_composite`, and shove it into `ffmpeg` to build the video.

In [27]:
def vidwrite(fn, timestamps, framerate=8, vcodec='libx264'):
    width, height = int(camera_image_width * camera_image_rescale), int(camera_image_height * camera_image_rescale)

    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgba', s=f'{width}x{height}')
            .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate)
            .overwrite_output()
            .run_async(pipe_stdin=True)
    )
    for t in sorted(timestamps):
        ci = camera_image_for_timestamp(frame_sources[t]['camera_time'])
        mi = mat_image_for_timestamp(frame_sources[t]['mat_time'])
        ci.alpha_composite(mi, dest=(37, 27))
        process.stdin.write(ci.tobytes())
    process.stdin.close()
    process.wait()

In [28]:
vidwrite(f'{outfile_name}.mp4', frame_sources)

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libope

In [29]:
vidwrite_camera(f'{outfile_name}_camera.mp4', camera_frames)

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libope

In [30]:
vidwrite_mat(f'{outfile_name}_mat.mp4', mat_frames)

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libope

In [31]:
camera_frames.dtype

dtype('float64')

In [32]:
mat_frames.dtype

dtype('float64')

In [34]:
camera_frame_number = 20
Image.fromarray((camera_frames[camera_frame_number] * 255).astype(np.uint8)).save(f'{outfile_name}_camera.png')

In [35]:
mat_frame_number = 20
Image.fromarray((mat_frames[mat_frame_number] * 255).astype(np.uint8)).save(f'{outfile_name}_mat.png')